In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [2]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/ESM/historic/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values
ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values

array([[[ 0.6764158 ,  0.8498988 ,  1.063465  , ...,  0.66933393,
          0.6382942 ,  0.6161675 ],
        [ 0.51208454,  0.6535262 ,  0.7983409 , ...,  0.37634483,
          0.39317453,  0.4287318 ],
        [ 0.27010962,  0.40374526,  0.4932245 , ..., -0.0439052 ,
          0.02290633,  0.13684982],
        ...,
        [-0.22322041, -0.21487327, -0.2063787 , ..., -0.25131282,
         -0.24205804, -0.23268627],
        [-0.1505498 , -0.14529137, -0.14001046, ..., -0.16692953,
         -0.16148826, -0.15602666],
        [-0.06991282, -0.06818231, -0.06645382, ..., -0.07489846,
         -0.07323544, -0.07157347]],

       [[ 0.17144182,  0.12738119,  0.21119103, ...,  0.44148993,
          0.35518792,  0.26787776],
        [-0.0761885 , -0.03999089,  0.02317043, ...,  0.04584048,
         -0.04192197, -0.0795617 ],
        [-0.36588094, -0.29935798, -0.2528414 , ..., -0.29041287,
         -0.37959164, -0.40252772],
        ...,
        [-0.01546755, -0.01054382, -0.00563325, ..., -

In [3]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [4]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(4810,)


In [5]:
AO_ds.to_netcdf('IITM-ESM-AOindex-NDJF-Daily-1980-2014.nc')

In [6]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(2.9542308, dtype=float32)
Coordinates:
    time     datetime64[ns] 2004-11-04T12:00:00

In [7]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [8]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-3.2310448, dtype=float32)
Coordinates:
    time     datetime64[ns] 2002-02-08T12:00:00

In [9]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [9]:
zLevData = xr.open_dataset('../../../Data/CMIP6/ESM/historic/zg_NDJFday_IITM-ESM_historical_r1i1p1f1_gn_18500101-20141231.nc')

In [10]:
zLevData.plev

<xarray.DataArray 'plev' (plev: 8)>
array([100000.,  85000.,  70000.,  50000.,  25000.,  10000.,   5000.,   1000.])
Coordinates:
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
Attributes:
    standard_name:  air_pressure
    long_name:      pressure
    units:          Pa
    positive:       down
    axis:           Z

In [11]:
z500=zLevData.zg.sel(plev=50000.0,method='nearest')
z500

<xarray.DataArray 'zg' (time: 4810, lat: 94, lon: 192)>
[86810880 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
  * lon      (lon) float64 0.0 1.875 3.75 5.625 7.5 ... 352.5 354.4 356.2 358.1
  * lat      (lat) float64 -88.54 -86.65 -84.75 -82.85 ... 84.75 86.65 88.54
    plev     float64 5e+04
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    units:          m
    comment:        Geopotential is the sum of the specific gravitational pot...
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2021-01-19T12:43:22Z altered by CMOR: Inverted axis: lat.

In [12]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [13]:
#z145E40N

In [14]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [15]:
#z20E55N

In [16]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [17]:
#z75E55N

In [18]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [19]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [20]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [21]:
EUVal

<xarray.DataArray 'zg' (time: 4810)>
array([ 0.03492972,  0.15465996,  0.2046555 , ..., -0.46875376,
       -0.45997501, -1.14519234])
Coordinates:
  * time       (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float64 144.4

In [22]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [23]:
EU_ds.to_netcdf('IITM-ESM-EUindex-NDJF-Daily-1980-2014.nc')